<a href="https://colab.research.google.com/github/entc-17-fyp-20/WPF_Data_Visualization/blob/Kusal/LSTM_RNN_model_for_short_term_power_forecasting_Tested_with_new_CEB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from collections import UserDict
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Image
%matplotlib inline
import plotly.express as px
import io


pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore")

url1 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/S6%2010%20seconed%20data.csv' #S6 10 seconed data.csv
url2 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/AE59_1.csv' #AE59_1
url3 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/AE59_2.csv' #AE59_2
url4 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/V08%20June%2010%20sec%20data.csv' #V08 June 10 sec data.csv; does not contain current columns
url5 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/V13%20June%2010%20sec%20data.csv' #V13 June 10 sec data.csv
url6 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/V08%20July%2010%20sec%20data.csv' #V08 July 10 sec data.csv
url7 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/V13%20July%2010%20sec%20data.csv' #V13 July 10 sec data.csv
url8 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/Mannar.csv'  #Mannar dataset-10_min-wind_speed.csv
url9 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/Mannar%20full%20data.csv' #Mannar full data set.csv
url10 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/2018%20Scada%20Data%20of%20a%20Wind%20Turbine%20in%20Turkey.csv' #2018 Scada Data of a Wind Turbine in Turkey - 10 min resolution data
url11 = 'https://raw.githubusercontent.com/entc-17-fyp-20/Datasets/main/CEB%20new%20updated(Only%20actual).csv' #CEB new dataset


**Train an validation data set**

In [ ]:
df = pd.read_csv(url9)
df=pd.DataFrame(df)
df

,Date & Time,Average Wind Speed (m/s),Average Wind Dirction (deg),Average Active Power (MW),Forecasted Power (kW)
0,8/1/2021 0:00,10.66,186.92,95.84,100.25
1,8/1/2021 0:10,10.71,188.15,96.03,100.01
2,8/1/2021 0:20,10.53,188.78,93.72,97.12
3,8/1/2021 0:30,10.20,189.80,88.84,96.96
4,8/1/2021 0:40,10.34,193.00,90.97,94.08
...,...,...,...,...,...
4404,8/31/2021 23:20,11.62,195.08,100.45,101.74
4405,8/31/2021 23:30,11.16,196.36,98.01,101.78
4406,8/31/2021 23:40,10.66,198.02,93.11,101.85
4407,8/31/2021 23:50,10.90,196.96,93.68,101.29


**Rename column names**

In [ ]:
col_1=(df.columns)[0]
col_2=(df.columns)[3]
feature = 'Active Power'
df.rename(columns={col_1 :'Date & Time' , col_2 : feature},inplace=True)
df


,Date & Time,Average Wind Speed (m/s),Average Wind Dirction (deg),Active Power,Forecasted Power (kW)
0,8/1/2021 0:00,10.66,186.92,95.84,100.25
1,8/1/2021 0:10,10.71,188.15,96.03,100.01
2,8/1/2021 0:20,10.53,188.78,93.72,97.12
3,8/1/2021 0:30,10.20,189.80,88.84,96.96
4,8/1/2021 0:40,10.34,193.00,90.97,94.08
...,...,...,...,...,...
4404,8/31/2021 23:20,11.62,195.08,100.45,101.74
4405,8/31/2021 23:30,11.16,196.36,98.01,101.78
4406,8/31/2021 23:40,10.66,198.02,93.11,101.85
4407,8/31/2021 23:50,10.90,196.96,93.68,101.29


**Set Data & Time as index**

In [ ]:
df=df.set_index('Date & Time')
df

,Average Wind Speed (m/s),Average Wind Dirction (deg),Active Power,Forecasted Power (kW)
Date & Time,,,,
8/1/2021 0:00,10.66,186.92,95.84,100.25
8/1/2021 0:10,10.71,188.15,96.03,100.01
8/1/2021 0:20,10.53,188.78,93.72,97.12
8/1/2021 0:30,10.20,189.80,88.84,96.96
8/1/2021 0:40,10.34,193.00,90.97,94.08
...,...,...,...,...
8/31/2021 23:20,11.62,195.08,100.45,101.74
8/31/2021 23:30,11.16,196.36,98.01,101.78
8/31/2021 23:40,10.66,198.02,93.11,101.85


**Divide train and validation sets**

In [ ]:
L=len(df)
valid_start=int(18*L/20)
train,valid=df[:valid_start],df[valid_start:]

print(train.shape)
print(valid.shape)


print(train.describe())
print(valid.describe())


(3968, 4)
(441, 4)
       Average  Wind Speed (m/s)   Average  Wind Dirction (deg)   \
count                   3,968.00                        3,968.00   
mean                        9.88                          196.16   
std                         2.65                           13.74   
min                         1.81                          130.84   
25%                         8.07                          188.48   
50%                        10.00                          194.03   
75%                        11.88                          202.26   
max                        15.88                          311.14   

       Active Power   Forecasted Power (kW)   
count      3,968.00                 3,968.00  
mean          71.39                    70.69  
std           31.76                    29.82  
min           -0.76                     1.83  
25%           45.82                    43.72  
50%           83.40                    76.94  
75%          100.97                   1

# **Data preparation**

In [ ]:
T = 18
HORIZON = 1

In [ ]:
scaler = MinMaxScaler()
train=pd.DataFrame(train[feature])
train[feature] = scaler.fit_transform(train)

train_shifted = train.copy()
train_shifted['y_t+1'] = train_shifted[feature].shift(-1)
train_shifted.head(10)

,Active Power,y_t+1
Date & Time,,
8/1/2021 0:00,0.93,0.93
8/1/2021 0:10,0.93,0.91
8/1/2021 0:20,0.91,0.86
8/1/2021 0:30,0.86,0.88
8/1/2021 0:40,0.88,0.90
8/1/2021 0:50,0.90,0.91
8/1/2021 1:00,0.91,0.94
8/1/2021 1:10,0.94,0.94
8/1/2021 1:20,0.94,0.92


**Creating input data to model (Getting T lagged vaules)**

In [ ]:
for t in range(1, T+1):
    train_shifted['active_power_t-'+str(T-t)] = train_shifted[feature].shift(T-t)
train_shifted = train_shifted.rename(columns={feature:'Original_active_power'})

# 4.Discard any samples with missing values
train_shifted = train_shifted.dropna(how='any')
train_shifted.head(10)

,Original_active_power,y_t+1,active_power_t-17,active_power_t-16,active_power_t-15,active_power_t-14,active_power_t-13,active_power_t-12,active_power_t-11,active_power_t-10,active_power_t-9,active_power_t-8,active_power_t-7,active_power_t-6,active_power_t-5,active_power_t-4,active_power_t-3,active_power_t-2,active_power_t-1,active_power_t-0
Date & Time,,,,,,,,,,,,,,,,,,,,
8/1/2021 2:50,0.90,0.89,0.93,0.93,0.91,0.86,0.88,0.90,0.91,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90
8/1/2021 3:00,0.89,0.87,0.93,0.91,0.86,0.88,0.90,0.91,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89
8/1/2021 3:10,0.87,0.85,0.91,0.86,0.88,0.90,0.91,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89,0.87
8/1/2021 3:20,0.85,0.82,0.86,0.88,0.90,0.91,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89,0.87,0.85
8/1/2021 3:30,0.82,0.79,0.88,0.90,0.91,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89,0.87,0.85,0.82
8/1/2021 3:40,0.79,0.73,0.90,0.91,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89,0.87,0.85,0.82,0.79
8/1/2021 3:50,0.73,0.70,0.91,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89,0.87,0.85,0.82,0.79,0.73
8/1/2021 4:00,0.70,0.71,0.94,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89,0.87,0.85,0.82,0.79,0.73,0.70
8/1/2021 4:10,0.71,0.72,0.94,0.92,0.93,0.94,0.95,0.97,0.97,0.97,0.93,0.90,0.89,0.87,0.85,0.82,0.79,0.73,0.70,0.71


**Train input and target arrays**

In [ ]:
y_train = np.array(train_shifted["y_t+1"])
X_train=np.array(train_shifted[['active_power_t-'+str(T-t) for t in range(1, T+1)]])

In [ ]:
# This is an important step to reshape the X into 3 dimension array
X_train = X_train.reshape(X_train.shape[0], T, 1)

In [ ]:
print(y_train.shape)
print(X_train.shape)

(3950,)
(3950, 18, 1)


**Data preparation - Validation data**

In [ ]:
valid=pd.DataFrame(valid[feature])
valid_new=df[valid_start-T+1:] #Add T values beforehand to the validation set
valid_new=pd.DataFrame(valid_new[feature])
valid_new.head(20)

,Active Power
Date & Time,
8/28/2021 19:50,102.40
8/28/2021 20:00,102.33
8/28/2021 20:10,102.47
8/28/2021 20:20,102.39
8/28/2021 20:30,102.28
8/28/2021 20:40,102.19
8/28/2021 20:50,102.68
8/28/2021 21:00,102.85
8/28/2021 21:10,102.70


In [ ]:
valid_new[feature]=scaler.transform(valid_new)

#Validataion set preparation(Lagged values,create input and target arrays,reshape)]

valid_shifted = valid_new.copy() 
valid_shifted['y_t+1'] = valid_shifted[feature].shift(-1)
valid_shifted.head(10)

for t in range(1, T+1):
    valid_shifted['active_power_t-'+str(T-t)] = valid_shifted[feature].shift(T-t)
vali_shifted = valid_shifted.rename(columns={feature:'Original_active_power'})

valid_shifted = valid_shifted.dropna(how='any')

y_valid = np.array(valid_shifted["y_t+1"])
X_valid=np.array(valid_shifted[['active_power_t-'+str(T-t) for t in range(1, T+1)]])
X_valid = X_valid.reshape(X_valid.shape[0], T, 1)

In [ ]:
print(y_valid.shape)
print(X_valid.shape)

(440,)
(440, 18, 1)


# **Implement the GRU-RNN**

In [ ]:
from keras.models import Model, Sequential
from keras.layers import LSTM,LeakyReLU,Dropout,Dense
from keras.callbacks import EarlyStopping

In [ ]:
UNITS = 5 # number of units in the RNN layer
BATCH_SIZE = 32 # number of samples per mini-batch
EPOCHS = 50 # maximum number of times the training algorithm will cycle through all samples

In [ ]:
model = Sequential()

#model.add(tf.keras.layers.LSTM(128,input_shape=(win_length,num_feature),return_sequences=True))
#model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
#model.add(tf.keras.layers.LSTM(64,return_sequences=True))
#model.add(tf.keras.layers.Dropout(0.1))
#model.add(tf.keras.layers.LSTM(32,return_sequences=True))
#model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
#model.add(tf.keras.layers.Dropout(0.1))
#model.add(tf.keras.layers.LSTM(32,return_sequences=False))
#model.add(tf.keras.layers.Dropout(0.1))
#model.add(tf.keras.layers.Dense(1))

model.add(LSTM(UNITS, input_shape=(T, 1))
#model.add(LeakyReLU(alpha=0.5))
#model.add(LSTM(64,return_sequences=True))
#model.add(Dropout(0.1))
#model.add(LSTM(32,return_sequences=True))
#model.add(LeakyReLU(alpha=0.5))
#model.add(Dropout(0.1))
#model.add(LSTM(32,return_sequences=False))
#model.add(Dropout(0.1))
model.add(Dense(HORIZON))

SyntaxError: ignored

In [ ]:
model.compile(optimizer='Adam', loss='mse')
model.summary()

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

In [ ]:
history = model.fit(X_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_valid, y_valid),
                    callbacks=[earlystop],
                    verbose=1)

In [ ]:
plot_df = pd.DataFrame.from_dict({'train_loss':history.history['loss'], 'val_loss':history.history['val_loss']})
plot_df.plot(logy=True, figsize=(10,10), fontsize=12)
plt.xlabel('epoch', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.show()

# **Evaluate the model**

**Load test data set**

In [ ]:
df_test = pd.read_csv(url11)
df_test=pd.DataFrame(df_test)

In [ ]:
df_test=df_test.drop(0)
df_test=df_test.head(200)

In [ ]:
col_1=(df_test.columns)[0]
col_2=(df_test.columns)[1]
feature = 'Active Power'
df_test.rename(columns={col_1 :'Date & Time' , col_2 : feature},inplace=True)

In [ ]:
col_3=(df_test.columns)[2]
df_test.rename(columns={col_3 :'Forecasted power'},inplace=True)

In [ ]:
df_test=df_test.set_index('Date & Time')
df_test = df_test.apply(pd.to_numeric)
df_test=df_test.div(1000)

In [ ]:
df_test

In [ ]:
scaler = MinMaxScaler()
test=pd.DataFrame(df_test[feature])
test[feature] = scaler.fit_transform(test)
test


In [ ]:
test_shifted = test.copy()
test_shifted['y_t+1'] = test_shifted[feature].shift(-1)

for t in range(1, T+1):
    test_shifted['active_power_t-'+str(T-t)] = test_shifted[feature].shift(T-t)
test_shifted = test_shifted.rename(columns={feature:'Original_active_power'})

forecasted_values=np.array(df_test['Forecasted power'])
test_shifted['Forecasted power'] = forecasted_values
test_shifted

In [ ]:
test_shifted = test_shifted.dropna(how='any')
y_test = np.array(test_shifted['Original_active_power'])
X_test=np.array(test_shifted[['active_power_t-'+str(T-t) for t in range(1, T+1)]])
X_test = X_test.reshape(X_test.shape[0], T, 1)
X_test

In [ ]:
print(y_test.shape)
print(X_test.shape)

In [ ]:
predictions = model.predict(X_test)
predictions=pd.DataFrame(predictions)
predictions

In [ ]:
eval_df = pd.DataFrame(predictions, columns=['t+'+str(t) for t in range(1, HORIZON+1)])
eval_df['Date & Time'] = test_shifted.index
eval_df = pd.melt(eval_df, id_vars='Date & Time', value_name='prediction', var_name='h')
eval_df['actual'] = np.transpose(y_test).ravel()
eval_df['prediction'] =predictions 

eval_df['Forecasted power']=np.array(test_shifted['Forecasted power'])
eval_df

In [ ]:
eval_df[['prediction']]=scaler.inverse_transform(np.array(eval_df[['prediction']]))
eval_df[['actual']]=scaler.inverse_transform(np.array(eval_df[['actual']]))
eval_df

In [ ]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

actual = eval_df['actual']
pred = eval_df['prediction']

mape(actual, pred)

In [ ]:
def rmse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    x=np.mean((actual - pred)**2)
    return np.sqrt(x)

actual = eval_df['actual']
pred = eval_df['prediction']

rmse(actual, pred)

In [ ]:
eval_df.plot(x='Date & Time', y=['prediction','actual'], style=['g','b'],fontsize=12, figsize=(15, 8))
plt.title("LSTM model:Expected vs predicted power forecasting",fontsize=16)
plt.xlabel('Date & Time', fontsize=16)
plt.ylabel('Active power (MW)', fontsize=16)
plt.grid()
plt.savefig("LSTM-results.pdf")
plt.show()

In [ ]:
eval_df.plot(x='Date & Time', y=['Forecasted power', 'actual'], style=['r', 'b'],fontsize=12, figsize=(15, 8))
plt.xlabel('Date & Time', fontsize=16)
plt.ylabel('Active power (MW)', fontsize=16)
plt.grid()
plt.show()